In [1]:
import numpy as np
import imageio
import cv2
import glob
import os

import open3d as o3d
from pyquaternion import Quaternion
import matplotlib.pyplot as plt

from MatlabTests.python.read_write_model import *
from MatlabTests.python.read_dense import *
from Bastian_utils.ShowUtils import *

# 3. TSDF using Floor repair results

In [2]:
worksapce_path = "./WinterThirdFloor/winter_third_floor_results/"
images_folder = worksapce_path + "/images/"
sub_folders = ['cut_'+str(i+1) for i in range(3)]
images_all = []
for sub_folder in sub_folders:
    images = glob.glob(images_folder+ sub_folder + "/*.png")
    images = rank_images(images)
    images_all = images_all + images
print("==> Loaded", len(images_all), "images")

==> Loaded 4419 images


In [3]:
input_model = "./WinterThirdFloor/winter_third_floor_results/sparse"
input_format = ".bin"
image_folder = "./WinterThirdFloor/winter_third_floor_results/images/"
#depth_folder = "garden/floor_process/fill_floor/"
#depth_folder_o3d = "./WinterThirdFloor/plane_completion/floor_o3d/"
depth_folder_o3d = "./WinterThirdFloor/tv_l2/floor_o3d/"
depth_folder_mine = "./WinterThirdFloor/plane_completion/floor_fill/"

In [4]:
cameras, images, points3D = read_model(path=input_model, ext=input_format)
print("num_cameras:", len(cameras))
print("num_images:", len(images))
print("num_points3D:", len(points3D))

num_cameras: 3
num_images: 4419
num_points3D: 347505


In [5]:
wrong_idx_o3d = {"cut_1": [36, 492, 493, 494],
                "cut_2" : [83, 119, 186, 187, 212, 273, 306, 350, 386, 403, 417, 418, 440, 445, 446, 451, 452, 453, 455,
                          468, 471, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 566, 567, 586,
                          589, 590, 591, 592, 608, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634,
                          635, 636, 637, 638, 639, 640, 652, 653, 658, 659, 660, 661, 662, 774, 775, 776, 777, 778, 779, 780,
                          806, 807, 808, 809, 810, 811, 870, 877, 905, 906, 1012, 1218, 1219, 1220, 1221, 1222, 1236, 1401],
                "cut_3" : [520, 521, 522,523,524,525,526,527,528,529,532,535,536,537,538,539,541,544,545,546,547,548,549,592,595,
                          596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,615,616,617,618,619,620,622,
                          1312,1444,1445,1446,1447,1448,1449,1450,1451,1452,1453,1454,1459,1499,1500,1516,1530,1531,1532,1539,1540,
                          1541,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1558,1560,1561,1562,1563,1564,1565,1566,1567,
                          1587,1605,1606,1607,1608,1609,1610,1611,1617,1618,1619,1620,1621,1622,1626,1627,1628,1629,1630,1631,1638,
                          1639,1640,1641,1642,1643,1644,1645,1650,1655,1662,1663,1664,1665,1670,1671,1673,1675,1676,1677,1678,
                          1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,1692,1693,1698,1706,1708,1709,1710,1711,1712,1713,1714,
                          1715,1722,1743,1744,1745,1746,1747,1749,1789,1790,1791,1792,1793,1794,1795,1796,1802,1806,1807,1814,1815,1816,
                          1817,1819,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832,1833,1834,1846,1865,1928,1929]}
wrong_idx_o3d["cut_3"] = wrong_idx_o3d["cut_3"] + np.arange(1929,2126).tolist()

wrong_idx_mine = {"cut_1": [51, 62, 120, 484, 487, 491, 494, 496, 505, 509, 512, 513, 546, 599, 605, 639, 646, 647],
                "cut_2" : [],
                "cut_3" : []}

### result o3d floor repair

In [6]:
volume = o3d.integration.ScalableTSDFVolume(
    voxel_length=1.00 / 512.0,
    sdf_trunc=0.02,
    color_type=o3d.integration.TSDFVolumeColorType.RGB8)

count = 1
for image_id, Image in images.items():
    pose_camera = q_t_to_matrix(Image.qvec, Image.tvec*0.1) #np.linalg.inv
    image_file = image_folder + "/" + Image.name
    depth_file = depth_folder_o3d + "/" + Image.name.split("/")[-2] + "/" + Image.name.split("/")[-1] + ".npy"
    #print(image_file, depth_file)
    
    image_id = int(Image.name[:-4].split("/")[-1])
    folder_name = Image.name[:-4].split("/")[0]
    if(not os.path.exists(depth_file)):
        continue
    if(image_id in wrong_idx_o3d[folder_name]):
        continue
    
    color = o3d.io.read_image(image_file)
    depth_py = np.load(depth_file)*100
    depth_buffer = depth_py.astype(np.float32).copy()
    depth = o3d.geometry.Image(depth_buffer)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
                color, depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
    
    camera_param = cameras[Image.camera_id].params
    volume.integrate(
        rgbd_image,
        o3d.camera.PinholeCameraIntrinsic(depth_py.shape[1], depth_py.shape[0], camera_param[0], camera_param[1], camera_param[2], camera_param[3]),
        pose_camera)
    print("\r current process frame :["+str(count)+"/"+str(len(images))+"]", end="", flush= True)
    count += 1

 current process frame :[3925/4419]

In [7]:
print("Extract a triangle mesh from the volume and visualize it.")
mesh = volume.extract_triangle_mesh()
mesh.compute_vertex_normals()
o3d.io.write_triangle_mesh("./WinterThirdFloor/o3d_mesh_detail.ply", mesh)
o3d.visualization.draw_geometries([mesh])

Extract a triangle mesh from the volume and visualize it.


In [11]:
o3d.visualization.draw_geometries([mesh])

In [10]:
o3d.io.write_triangle_mesh("./WinterThirdFloor/o3d_mesh.ply", mesh)

True

![test mesh](./WinterThirdFloor/mesh_screen_shot_2/002.png)